<a href="https://colab.research.google.com/github/cedro3/PaintTransformer/blob/main/Paint_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

In [ ]:
# githubからコードを取得
!git clone https://github.com/cedro3/PaintTransformer.git
%cd PaintTransformer/inference

# プロセス動画作成

In [ ]:
# 画像指定
input_img = 'yui.jpg'

In [ ]:
# --- ペイントプロセスの作成 ---
from inference import run_inference
import os 
import shutil

# outputフォルダーリセット
if os.path.isdir('output'):
    shutil.rmtree('output')

# name取得
name = input_img[:-4]

# プロセス作成
run_inference(input_path='input/'+input_img,
              model_path='model.pth',
              output_dir='output/', # whether need intermediate results for animation.
              need_animation=True,  # resize original input to this size. None means do not resize.
              resize_h=None,        # resize original input to this size. None means do not resize.
              resize_w=None,
              serial=True)          # if need animation, serial must be True.

In [ ]:
# --- プロセスからgifとmp4を作成 ---
import glob
from PIL import Image

# Set to dir with output images
in_dir = 'output/'+name+'/*.jpg'
out_path = 'output/process.gif'

img, *imgs = [Image.open(f) for f in sorted(glob.glob(in_dir))]
img.save(fp=out_path, format='GIF', append_images=imgs,
          save_all=True, duration=100, loop=0)

! ffmpeg -i output/process.gif -pix_fmt yuv420p output/process.mp4

In [ ]:
# --- mp4の再生 ---
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output/process.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="70%" height="70%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

# ペイント動画作成

In [ ]:
# --- ビデオを静止画にバラす ---
import os
import shutil
import cv2
import math

# imagesフォルダーリセット
if os.path.isdir('images'):
    shutil.rmtree('images')
os.makedirs('images', exist_ok=True)
 
def video_2_images(video_file= './input/yui.mp4',   # ビデオ指定
                   image_dir='./images/', 
                   image_file='%s.jpg'):  
 
    # Initial setting
    i = 0
    interval = 3
    length = 120  # 最大フレーム数
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()  
    print('images_number = ', math.ceil(i/interval))

def main():
    video_2_images()
    
if __name__ == '__main__':
    main()

In [ ]:
# --- 静止画を油絵に変換する ---
from inference import run_inference
import os
import shutil
import glob
from tqdm import tqdm

# outputフォルダーリセット
if os.path.isdir('frame'):
    shutil.rmtree('frame')

files = glob.glob('./images/*.jpg')
files.sort()

for file in tqdm(files):
    run_inference(input_path=file,
         model_path='model.pth',
         output_dir='frame/',
         need_animation=False,  # whether need intermediate results for animation.
         resize_h=None,         # resize original input to this size. None means do not resize.
         resize_w=None,         # resize original input to this size. None means do not resize.
         serial=True)          # if need animation, serial must be True.

In [ ]:
# --- mp4作成 ---
# output.mp4をリセット
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

# frameフォルダーの画像をmp4に変換
!ffmpeg -r 10 -i frame/%06d.jpg -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
# --- mp4再生 ---
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="70%" height="70%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")